In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sb
import numpy as np
import os
import sys
import warnings 
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import logging
logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',level=logging.INFO)
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator,ClassifierMixin,TransformerMixin,clone
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
import xgboost as xgb
from sklearn.ensemble import VotingClassifier

In [2]:
cwd=os.getcwd()
cwd

'C:\\Users\\perma\\Documents\\GitHub\\Kaggle_Titanic\\code'

In [3]:
#split training set into train_1 which consists of all the survival,train_0 which consists of all the dead
train_path=os.path.join(os.path.dirname(cwd),"data/train_raw.csv")
test_path=os.path.join(os.path.dirname(cwd),"data/test_raw.csv")
train=pd.read_csv(train_path,header=0)
test=pd.read_csv(test_path,header=0)
train_test={"train":train,"test":test}

In [4]:
#get Family_Size
for key in train_test:
    data=train_test[key]
    data["Family_Size"]=data['SibSp']+data["Parch"]+1

In [5]:
train_and_test=pd.concat([train,test],ignore_index=True)

In [6]:
family_survival=train[["Ticket","Survived"]].groupby(by=["Ticket"]).mean()
train_and_test["Family_Survival"]="median"
for index in train_and_test.index:
    if(train_and_test.loc[index,"Family_Size"]>1):
        ticket=train_and_test.loc[index,"Ticket"]
        if(ticket in family_survival.index):
            survival=family_survival.loc[ticket]
            if(survival.values>0.5):
                train_and_test.loc[index,"Family_Survival"]="high"
            elif(survival.values<0.5):
                train_and_test.loc[index,"Family_Survival"]="low"

In [7]:
train["Family_Survival"]=train_and_test.loc[0:890,"Family_Survival"].values
test["Family_Survival"]=train_and_test.loc[891:,"Family_Survival"].values
def get_dummies(df,*features):
    logging.info("---------------get_dummies-------------------")
    for feature in features:
        logging.info("{}:start".format(feature))
        dummies=pd.get_dummies(df[feature],prefix=feature)
        df=df.join(dummies)
        df.drop(feature,inplace=True,axis=1)
        logging.info("{}:done".format(feature))
    return df       

In [8]:
feature_to_dummies=["Sex","Embarked","Family_Survival"]
for key in train_test:
    data=train_test[key]
    data=get_dummies(data,*feature_to_dummies)
    train_test[key]=data

2018-11-18 12:08:09,187:INFO:---------------get_dummies-------------------
2018-11-18 12:08:09,191:INFO:Sex:start
2018-11-18 12:08:09,196:INFO:Sex:done
2018-11-18 12:08:09,196:INFO:Embarked:start
2018-11-18 12:08:09,201:INFO:Embarked:done
2018-11-18 12:08:09,202:INFO:Family_Survival:start
2018-11-18 12:08:09,207:INFO:Family_Survival:done
2018-11-18 12:08:09,208:INFO:---------------get_dummies-------------------
2018-11-18 12:08:09,208:INFO:Sex:start
2018-11-18 12:08:09,213:INFO:Sex:done
2018-11-18 12:08:09,213:INFO:Embarked:start
2018-11-18 12:08:09,218:INFO:Embarked:done
2018-11-18 12:08:09,219:INFO:Family_Survival:start
2018-11-18 12:08:09,223:INFO:Family_Survival:done


In [9]:
features=[  'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',  'Family_Size', 'Sex_female', 'Sex_male',\
          'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Family_Survival_high',\
       'Family_Survival_low', 'Family_Survival_median']

In [10]:
X=train_test["train"][features].values
y=train_test["train"]["Survived"].values
X.shape
y.shape

(891, 14)

(891,)

In [11]:
from random import shuffle

In [12]:
train0_index=train.loc[train["Survived"]==0].index
train1_index=train.loc[train["Survived"]==1].index

In [13]:
# how many times we are gonna do this
balanced_resample_time=20
sample_sets=[]
for i in range(balanced_resample_time):
    index1,index2=train_test_split(train0_index,train_size=350,shuffle=True)
    train_index=np.hstack((index1,train1_index))
    shuffle(train_index)
    sample_sets.append(train_index)

In [14]:
class Voter:
    def __init__(self):
        logging.info("A Voter has been created")
    def vote(self,array):
        array=np.array(array)
        return np.argmax(np.bincount(array))
        

In [15]:
n_estimators=10

In [24]:
for sample_set in sample_sets:
    rf=RandomForestClassifier(10)
    bg=GradientBoostingClassifier(n_estimators=10)
    xg=xgb.XGBClassifier(10)
    votingclassifier=VotingClassifier([('rf',rf),('bg',bg),("xg",xg)])
    scores=cross_val_score(estimator=votingclassifier,X=X[sample_set],y=y[sample_set],cv=10)
    "score:{}".format(np.mean(scores))

'score:0.8510973084886129'

'score:0.8641614906832299'

'score:0.851055900621118'

'score:0.8684679089026914'

'score:0.8626293995859212'

...

'score:0.8526293995859213'

'score:0.8598136645962733'

'score:0.8541614906832298'

'score:0.8554658385093168'

'score:0.8597101449275362'

# conlusion
although this is  not a balanced dataset for 549 dead cases and 342 survivals
but after my exploration on this issue ,i found that balanced dataset in which each class has 50 percent doesnt help improving the 
algorithm performance.